In [1]:
import pandas as pd
import numpy as np
import yfinance as yf
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

# Fetch data
df = yf.download('AAPL', start='2010-01-01', end='2023-06-29')

# Scale the data with MinMaxScaler
scaler = MinMaxScaler()
df_scaled = pd.DataFrame(scaler.fit_transform(df), columns=df.columns, index=df.index)

# Add a column for previous day's close price
df_scaled['Prev_Close'] = df_scaled['Close'].shift(1)

# Drop the first row, which will contain NaN because of the shift operation
df_scaled = df_scaled.dropna()

# Separate features (X) and target (y)
X = df_scaled.drop('Close', axis=1)
y = df_scaled['Close']

# Split into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05, random_state=42)

# Now, you can use these train and test datasets in your neural network model


[*********************100%***********************]  1 of 1 completed


In [2]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, BatchNormalization
from keras.optimizers import RMSprop
from keras.regularizers import l1
from keras.callbacks import EarlyStopping
from keras.layers import LSTM


nn = Sequential()
nn.add(Dense(128, input_shape=(X_train.shape[1],), kernel_initializer='normal', activation='relu'))  
nn.add(BatchNormalization())
nn.add(Dropout(0.3))
nn.add(Dense(64, kernel_initializer='normal', activation='relu', kernel_regularizer=l1(0.001)))
nn.add(BatchNormalization())
nn.add(Dropout(0.3))
nn.add(Dense(32, kernel_initializer='normal', activation='relu', kernel_regularizer=l1(0.001)))
nn.add(Dense(1, kernel_initializer='normal'))

optimizer = RMSprop(learning_rate=0.0001)
nn.compile(loss='mean_absolute_error', optimizer=optimizer)

early_stopping = EarlyStopping(monitor='val_loss', patience=30, verbose=1)

nn.fit(X_train, y_train, epochs=200, batch_size=32, validation_split=0.2, callbacks=[early_stopping])


NameError: name 'timesteps' is not defined

In [11]:
# Make predictions on the test data points
scaled_predictions = nn.predict(X_test)

# Reshape the scaled_predictions array to match the expected shape
scaled_predictions = scaled_predictions.reshape(-1, 1)

# Reverse the scaling of the predictions
# We only use the scaler fitted on 'Close' prices, so we have to extract that scaler
close_scaler = MinMaxScaler()
close_scaler.min_, close_scaler.scale_ = scaler.min_[df.columns.get_loc('Close')], scaler.scale_[df.columns.get_loc('Close')]

unscaled_predictions = close_scaler.inverse_transform(scaled_predictions)

# Convert the unscaled predictions to a DataFrame with appropriate index
predictions_df = pd.DataFrame(unscaled_predictions, columns=['Predicted Price'], index=X_test.index)

print(predictions_df)


6/6 [==============================] - 0s 655us/step
            Predicted Price
Date                       
2011-03-02        10.912245
2013-12-19        18.299976
2018-04-26        40.723755
2011-09-21        12.868277
2011-11-28        11.850340
...                     ...
2014-11-17        26.600733
2014-03-17        17.483988
2018-12-10        41.133907
2015-10-26        27.011459
2017-03-27        33.605240

[170 rows x 1 columns]


In [12]:
X_train.head(100)

,Open,High,Low,Adj Close,Volume,Prev_Close
Date,,,,,,
2021-10-14,0.746934,0.748387,0.745085,0.744073,0.018805,0.734965
2023-05-09,0.917817,0.910552,0.911507,0.903406,0.005489,0.913647
2010-04-30,0.015177,0.014561,0.013975,0.011426,0.274822,0.014997
2021-12-29,0.952502,0.949317,0.947679,0.937723,0.014711,0.945392
2013-06-21,0.044603,0.043740,0.043032,0.038119,0.241587,0.044016
...,...,...,...,...,...,...
2018-01-22,0.206864,0.204729,0.206605,0.196954,0.039678,0.207006
2021-07-16,0.782006,0.780536,0.769255,0.757085,0.031453,0.776469
2016-03-22,0.107380,0.108379,0.107894,0.101715,0.051242,0.107563


In [13]:
X_test.head(30)

,Open,High,Low,Adj Close,Volume,Prev_Close
Date,,,,,,
2011-03-02,0.031085,0.030920,0.031239,0.026491,0.307397,0.030793
2013-12-19,0.070444,0.069124,0.069823,0.061406,0.154466,0.070242
2018-04-26,0.188665,0.188259,0.188312,0.181304,0.041530,0.186706
2011-09-21,0.044829,0.044050,0.043802,0.036424,0.309231,0.043353
2011-11-28,0.035501,0.035289,0.035571,0.030463,0.168608,0.033586
2016-06-14,0.096430,0.096337,0.096197,0.090874,0.050131,0.095816
2011-12-29,0.041626,0.040938,0.041532,0.035262,0.097942,0.041236
2012-12-06,0.066389,0.069771,0.064865,0.059586,0.618710,0.067896
2021-01-13,0.673202,0.680426,0.673074,0.671311,0.028953,0.668569


In [14]:
predictions_df.head(20)

,Predicted Price
Date,
2011-03-02,10.912245
2013-12-19,18.299976
2018-04-26,40.723755
2011-09-21,12.868277
2011-11-28,11.850340
2016-06-14,23.638708
2011-12-29,12.971745
2012-12-06,17.841825
2021-01-13,125.990585


In [15]:
df.head(15)

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2010-01-04,7.622500,7.660714,7.585000,7.643214,6.496294,493729600
2010-01-05,7.664286,7.699643,7.616071,7.656429,6.507527,601904800
2010-01-06,7.656429,7.686786,7.526786,7.534643,6.404016,552160000
2010-01-07,7.562500,7.571429,7.466071,7.520714,6.392174,477131200
2010-01-08,7.510714,7.571429,7.466429,7.570714,6.434671,447610800
2010-01-11,7.600000,7.607143,7.444643,7.503929,6.377911,462229600
2010-01-12,7.471071,7.491786,7.372143,7.418571,6.305360,594459600
2010-01-13,7.423929,7.533214,7.289286,7.523214,6.394300,605892000
2010-01-14,7.503929,7.516429,7.465000,7.479643,6.357267,432894000


In [16]:
import yfinance as yf
import pandas as pd
ticker_symbol = "AAPL"
# Fetch the live data
def fetch_live_data(ticker_symbol):
    # Get the last month's worth of data (modify this as required)
    data = yf.download(ticker_symbol, period="1mo")
    # Add a column for previous day's adjusted close price
    data['Prev_Adj_Close'] = data['Adj Close'].shift(1)
    # Drop the first row, which will contain NaN because of the shift operation
    data = data.dropna()
    return data

# Specify the ticker symbol of the stock we want to predict
X_columns = ['Open', 'High', 'Low','Close', 'Adj Close', 'Volume']

# Fetch the live data
live_data = fetch_live_data(ticker_symbol)

# Assume you've defined `X_columns` somewhere, which includes all feature columns
# you used for your training data. This is important to maintain consistency in
# the number of features you feed to your trained model.

# Pre-process the live data
X_live = live_data[X_columns]
scaled_live_data = scaler.transform(X_live)

# Use the model to make predictions
scaled_predictions = nn.predict(scaled_live_data)

# Post-process the predictions
unscaled_predictions = close_scaler.inverse_transform(scaled_predictions)

# Use or display the predictions
print(unscaled_predictions)


[*********************100%***********************]  1 of 1 completed
1/1 [==============================] - 0s 8ms/step
[[128.45709]
 [128.60661]
 [129.75278]
 [130.20078]
 [130.58556]
 [130.65973]
 [131.40523]
 [131.82106]
 [131.3792 ]
 [130.96375]
 [131.44978]
 [131.87878]
 [131.9314 ]
 [132.2131 ]
 [132.9695 ]
 [133.33899]
 [134.85434]
 [134.79308]
 [134.2516 ]]


In [17]:
# Pre-process the live data
X_live = live_data[X_columns]
scaled_live_data = scaler.transform(X_live)

# Use the model to make predictions
scaled_predictions = nn.predict(scaled_live_data)

# Post-process the predictions
unscaled_predictions = close_scaler.inverse_transform(scaled_predictions)

# Use or display the predictions
print(unscaled_predictions)


1/1 [==============================] - 0s 8ms/step
[[128.45709]
 [128.60661]
 [129.75278]
 [130.20078]
 [130.58556]
 [130.65973]
 [131.40523]
 [131.82106]
 [131.3792 ]
 [130.96375]
 [131.44978]
 [131.87878]
 [131.9314 ]
 [132.2131 ]
 [132.9695 ]
 [133.33899]
 [134.85434]
 [134.79308]
 [134.2516 ]]


In [20]:
# Use the model to make predictions
scaled_predictions = nn.predict(scaled_live_data)

# Post-process the predictions
unscaled_predictions = close_scaler.inverse_transform(scaled_predictions)

# Convert the unscaled predictions to a DataFrame with dates
predictions_df = pd.DataFrame(unscaled_predictions, columns=['Predicted Price'], index=X_live.index)

print(predictions_df)


1/1 [==============================] - 0s 8ms/step
            Predicted Price
Date                       
2023-06-07       128.457092
2023-06-08       128.606613
2023-06-09       129.752777
2023-06-12       130.200775
2023-06-13       130.585556
2023-06-14       130.659729
2023-06-15       131.405228
2023-06-16       131.821060
2023-06-20       131.379196
2023-06-21       130.963745
2023-06-22       131.449783
2023-06-23       131.878784
2023-06-26       131.931396
2023-06-27       132.213104
2023-06-28       132.969498
2023-06-29       133.338989
2023-06-30       134.854340
2023-07-03       134.793076
2023-07-05       134.251602


In [19]:
print(live_data)

                  Open        High         Low       Close   Adj Close   
Date                                                                     
2023-06-07  178.440002  181.210007  177.320007  177.820007  177.820007  \
2023-06-08  177.899994  180.839996  177.460007  180.570007  180.570007   
2023-06-09  181.500000  182.229996  180.630005  180.960007  180.960007   
2023-06-12  181.270004  183.889999  180.970001  183.789993  183.789993   
2023-06-13  182.800003  184.149994  182.440002  183.309998  183.309998   
2023-06-14  183.369995  184.389999  182.020004  183.949997  183.949997   
2023-06-15  183.960007  186.520004  183.779999  186.009995  186.009995   
2023-06-16  186.729996  186.990005  184.270004  184.919998  184.919998   
2023-06-20  184.410004  186.100006  184.410004  185.009995  185.009995   
2023-06-21  184.899994  185.410004  182.589996  183.960007  183.960007   
2023-06-22  183.740005  187.050003  183.669998  187.000000  187.000000   
2023-06-23  185.550003  187.559998  18